In [1]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

from RFSoC_Daq import RFSoC_Daq
from Biquad import Biquad
from SimBiquad import SimBiquad
from Biquad_Daq import Biquad_Daq
from Filter import Filter

from Waveforms.Waveform import Waveform

In [2]:
daq = Biquad_Daq()
sim = SimBiquad()
filter = Filter()
filter.calc_params(460)

DEBUG:RFSoC_Daq:Asked to load overlay at /home/xilinx/python/zcumts.py
DEBUG:RFSoC_Daq:Adding directory /home/xilinx/python to module search path
DEBUG:RFSoC_Daq:Changing directory to /home/xilinx/python
DEBUG:RFSoC_Daq:Going to try to import zcumts
DEBUG:RFSoC_Daq:Found Overlay class Overlay from module pynq.overlay
DEBUG:RFSoC_Daq:Found RFSoC overlay zcuMTS


Turning on SYNC
Turning off SYNC


DEBUG:RFSoC_Daq:Created RFSoC device
DEBUG:RFSoC_Daq:Restoring original module search path
DEBUG:RFSoC_Daq:Going back to original directory /home/xilinx/rfsoc-pydaq/Biquad/Notebooks/tests


In [3]:
import pynq

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [4]:
def update_plot():
    
    global daq_output
    global sim_output
    global diff_output
    
    Xn = [0, 0, 0, 0, 0, 0, 0]
    Cross = [1, 0, 0, 0]
    C = [0 ,0, 0, 0]
    
    daq.update_params(**filter.get_params())
    
    # daq.set_Xn(*Xn)
    # daq.set_poleFIR(*Cross)
    # daq.set_poleIIR(*C)
    
    daq.quantise_coeffs()
    
    daq.run_zeroFIR()
    daq.run_poleFIR()
    daq.run_poleIIR()
    
    daq.update_waveforms()
    
    sim.update_params(**filter.get_params())
    
    sim.data = daq.adcBuffer[0] >> 4
    
    # sim.set_Xn(*Xn)
    # sim.set_poleFIR(*Cross)
    # sim.set_poleIIR(*C)
    
    sim.quantise_coeffs()
    sim.run_zeroFIR(quant=False)
    sim.run_poleFIR(quant=True)
    sim.run_poleIIR(quant=True)
    
    daq_output = daq.extract_biquad()
    sim_output = sim.extract_biquad(quant=False)
    diff_output = Waveform(daq_output.waveform - sim_output.waveform)
    
    fig, (axB,axS,axR) = plt.subplots(3, 1, figsize=(25, 15))
        
    daq_output.plotWaveform(axB, title="Biquaded ADC224_T0_CH0")
    
    sim_output.plotWaveform(axS, title="Simulated Biquad (No Quantisation)")
    
    diff_output.plotWaveform(axR, title="Difference")
    
    axR.axhline(y=4, color='r', linestyle='--', linewidth=1, label="Error Allowance")
    axR.axhline(y=-4, color='r', linestyle='--', linewidth=1)
    
    axR.text(0.93, 0.97, filter.__str__(), verticalalignment='top', horizontalalignment='right',
            transform=axR.transAxes, bbox=dict(facecolor='white', alpha=0.5))
    
    plt.show()
    
    
    # print(f'Raw ADC Output : {sim.extract_raw()[0]}')
    # print(f'Basic Prediction (A*Value) : {filter.A*sim.extract_raw()[0]}')
    # print(f'Daq Biquad Output : {daq_output[0]}')
    # print(f'Sim Biquad Output : {sim_output[0]}')

In [5]:
A_slider = widgets.FloatSlider(min=-4, max=4, step=0.05, value=filter.A, description='A:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=1, step=0.05, value=filter.P, description='P:', continuous_update=False)
Zero_slider = widgets.FloatSlider(min=200, max=1200, step=5, value=filter.fz, description='Zero:', continuous_update=False)
Pole_slider = widgets.FloatSlider(min=1, max=1500, step=5, value=filter.fp, description='Pole:', continuous_update=False)

plot_output = widgets.Output()

def update_params(_=None):
    with plot_output:
        plot_output.clear_output(wait=True)
        filter.fz = Zero_slider.value
        filter.A = A_slider.value
        filter.P = P_slider.value
        filter.fp = Pole_slider.value
        update_plot()

def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        if change['owner'] == Zero_slider:
            filter.fz = Zero_slider.value
            A_slider.value = filter.A
            P_slider.value = filter.P
            Pole_slider.value = filter.fp
        update_plot()
        
update_button = widgets.Button(description="Update Plot")
update_button.on_click(update_params)
Zero_slider.observe(update, names='value')

slider_box = widgets.HBox([A_slider, P_slider, Zero_slider, Pole_slider, update_button])
ui = widgets.VBox([slider_box, plot_output])
display(ui)
update_params()

In [6]:
# import time
# start_time = time.time()

count = 0
err_one = 0
err_two = 0
big = 0

iterations = 100

daq.update_params(**filter.get_params())
daq.quantise_coeffs()
sim.update_params(**filter.get_params())
sim.quantise_coeffs()

daq.run_zeroFIR()
daq.run_poleFIR()
daq.run_poleIIR()

for _ in range(iterations):

    daq.update_waveforms()
    
    sim.data = daq.adcBuffer[0] >> 4
    sim.run_zeroFIR(quant=False)
    sim.run_poleFIR(quant=True)
    sim.run_poleIIR(quant=True)
    
    daq_output = daq.extract_biquad()
    sim_output = sim.extract_biquad(quant=False)
    diff_output = Waveform(daq_output.waveform - sim_output.waveform)
    
##For when we've got qunatisation    
#     if diff_output.calc_rms() != 0:
#         count+=1

#         if diff_output.calc_rms()==0.04419417382415922:
#             err_one+=1
#         if diff_output.calc_rms()==0.0625:
#             err_two+=1
#         if diff_output.calc_rms()>0.0625:
#             big +=1

##For when we don't have quantisation, i.e. allowance of 4
    if diff_output.calc_rms() > np.sqrt(2):
        count+=1


print('')
print(f"Total Error Count : {count}")
print(f"Total Error Percentage : {100*count/iterations}%")

# print('')
# print(f"1 sample 1 off Count : {err_one}")
# print(f"1 sample 1 off Percentage : {100*err_one/iterations}%")

# print('')
# print(f"1/2 sample 2/1 off Count : {err_two}")
# print(f"1/2 sample 2/1 off Percentage : {100*err_two/iterations}%")

# print('')
# print(f"Effective Error Count : {big}")
# print('')
# print(f"Effective Error Percentage : {100*big/iterations}%")

# end_time = time.time()
# print(f"\nTime taken: {end_time - start_time} seconds")


Total Error Count : 0
Total Error Percentage : 0.0%
